# Imports and settings

In [22]:
from openmm.app import *
from openmm import *                    
from openmm.unit import *

# Setup a simulation system + state + CVForces

In [23]:
# Load an already solvated PDB file and set up the system + state
pdb = PDBFile("../villin.pdb")
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology,
                                         nonbondedMethod=PME,
                                         nonbondedCutoff=10.0 * angstrom,
                                         constraints=HBonds,
                                         rigidWater=True,
                                         hydrogenMass=4.0 * amu)


## Set important distances to track during AMD (usefulf for analysis)

In [24]:
# Define three atom indices - these will be used to measure useful distances 
d1_atom1_ind = 83
d1_atom2_ind = 151
d2_atom2_ind = 254

Ultimately we want to track the value of these distances (lets call them D1 and D2), \
as a function of the free energy boost applied. \
To track D1 and D2, we could use a whole `Reporter` object to save the whole trajectory and measure it afterwards.\

A lightweight way of doing this without saving the whole trajectory is to use a `customCVForce` object set to 0 (i.e. no bias). This will compute the value of a CV at each step of the simulation.\
We then pass D1 and D2 as bondForces to the `customCVForce` object (using `r` to track their distance)

In [25]:
# define a distance measurer
dist_measurer = CustomCVForce("0")
# Create our two distances as separate Bond Forces and add them
# Note: If we added both D1 and D2 to the same BondForce, then doing 
#     dist_measurer.getCollectiveVariableValues will return only the first distance (D1)
# Hence we define them as separate BondForces
D1 = CustomBondForce("r")
D1.addBond(d1_atom2_ind, d1_atom1_ind)
D2 = CustomBondForce("r")
D2.addBond(d2_atom2_ind, d1_atom1_ind)

# Add each BondForce as CVs into the dist_measurer
dist_measurer.addCollectiveVariable("D1", D1)
dist_measurer.addCollectiveVariable("D2", D2)
system.addForce(dist_measurer)

5

# Setup Accelerated Molecular Dynamics with a Langevin Integrator

To set up an Accelerated MD (AMD) simulation in OpenMM, we could use the default AMD object.\
However this uses a Verlet integrator rather than Langevin dynamics.\
Luckily, OpenMM gives us the tools to write this ourselves, using the `CustomIntegrator` object!  

For Langevin-AMD, we're effectively adding a boost potential to the simulation. 
This boosted potential botential takes the form of\
$V'(r) = V_{0}(r) + \frac{(E-V(r))^2}{(\alpha+E-V_0(r))}$ 
where $V_0(r)$ is the initial potential energy\
and $\alpha$ and E are hyper parameters defining the potential.\
For further details on values to choose, see Hamelberg et al., J. Chem. Phys. 127, 155102 (2007).

Let's first define a `customIntegrator` to run this. \
Note that in our `customIntegrator` we are adding $V'(r)$ as a Force `fprime`.\
To derive this yourself, remember that a Force is the derivative of Energy with respect to position:\
$f_{prime} = - \frac{\delta V(r)}{\delta r}$  and that 
$V(r) = V_0 + V'(r)$

## Let's define our custom integrator

In [26]:
kB = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

import numpy as np
class LangevinAMDIntegrator(CustomIntegrator):
    """AMDIntegrator implements the aMD integration algorithm.
    The system is integrated based on a modified potential.  Whenever the energy V(r) is less than a
    cutoff value E, the following effective potential is used:
    V*(r) = V(r) + (E-V(r))^2 / (alpha+E-V(r))
    For details, see Hamelberg et al., J. Chem. Phys. 127, 155102 (2007).
    """

    def __init__(self, dt, alpha, E, temperature=300*kelvin, collision_rate=1.0/picosecond):
        """Create an AMDIntegrator.
        Parameters
        ----------
        dt : time 
            The integration time step to use (simtk quantity)
        alpha : energy
            The alpha parameter to use
        E : energy
            The energy cutoff to use
        temperature : temperature
            temperature of the system
        collision rate : collision rate
            Collision for the thermostat to update on
        """
        gamma = collision_rate*picoseconds # add this because SWIG
        CustomIntegrator.__init__(self, dt)
        # aMD boost parameters
        self.addGlobalVariable("alpha", alpha)
        self.addGlobalVariable("E", E)
        self.addPerDofVariable("oldx", 0)
        self.addGlobalVariable("deltaV", 0)
        self.addGlobalVariable("V0", 0)
        self.addPerDofVariable("sigma", 0)
        self.addUpdateContextState();
        
        # new variables for Langevin kernel
        self.addGlobalVariable('kT', kB * temperature)
        self.addComputePerDof("sigma", "sqrt(kT/m)")
        self.addGlobalVariable("a", np.exp(-1 * gamma)) #vscale?
        self.addGlobalVariable("b", np.sqrt(1 - np.exp(-2 * gamma))) # noise-scale?

        # Compute steps for Langevin - attempt at VRORV splitting?
        # The below dof_string lines are comments on how to modify fprime (thinking out loud basically)
        # dof_string+= "v + (dt / 2) * fprime/m;" # standard v updating for VRORV (I think)
        # dof_string+="fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2);" # the right hand addition is the secret amd sauce
        # dof_string = "modify=step(E-energy);" # this is defining the modify scaling

        # Put the above together into a single new -line for computing v
        self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) +   modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # langevin like normal
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx)/(dt / 2)")
        self.addComputePerDof("v", "(a * v) + (b * sigma * gaussian)") # taken from openmm-tutorial on custom integrators (link 1 above)
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx) / (dt / 2)")
        # Use the same line as what you calculated above
        self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # compute the V and deltaV values
        self.addComputeGlobal("V0", "energy")
        self.addComputeGlobal("deltaV","modify*(E-energy)^2/(alpha+E-energy); modify=step(E-energy)")
        self.addConstrainVelocities()

    def getAlpha(self):
        """Get the value of alpha for the integrator."""
        return self.getGlobalVariable(0)*kilojoules_per_mole

    def setAlpha(self, alpha):
        """Set the value of alpha for the integrator."""
        self.setGlobalVariable(0, alpha)

    def getE(self):
        """Get the energy threshold E for the integrator."""
        return self.getGlobalVariable(1)*kilojoules_per_mole

    def setE(self, E):
        """Set the energy threshold E for the integrator."""
        self.setGlobalVariable(1, E)

    def getEffectiveEnergy(self, energy):
        """Given the actual potential energy of the system, return the value of the effective potential."""
        alpha = self.getAlpha()
        E = self.getE()
        if not is_quantity(energy):
            energy = energy*kilojoules_per_mole # Assume kJ/mole
        if (energy > E):
            return energy
        return energy+(E-energy)*(E-energy)/(alpha+E-energy)


To choose your own E and $\alpha$ values, the "best practice" is to run an Eq. MD simulation\
for ~10 nanoseconds, and look at the distribution of potential energy values.\
You'll want to set $E=<V(r)>$ of the V(r) distribution and $\alpha$ to some fraction of E.\
Typically $\alpha=0.2V(r)$ is a reasonable starting point 

Now let's define the integrator object using an $\alpha$ and $E$ based on some previous simulations:

In [27]:
lang_amd_integrator = LangevinAMDIntegrator(dt=0.002 * picosecond,
                                            alpha=13060*kilojoules_per_mole,
                                            E=-63500*kilojoules_per_mole,
                                            temperature=300*kelvin,
                                            collision_rate=1.0/picosecond)

In [28]:
lang_amd_simulation = Simulation(pdb.topology, system, lang_amd_integrator)

With AMD, you want to track the boost potential applied at each frame.\
From this set of boosts you can reconstruct the effective FES explored.

You'll want to project the FES on some features you care about \
(We defined ours so with the `dist_measurer` above). \
We can directly extract both the total energy (for comparison sake), and the \
boost potential (which is what we really care about) from the integrator directly.\
We can also extract our CVs directly thanks to the `dist_measurer` and print them all out.

In [29]:
# Now run the simulation, tracking the boost using the Integrator
lang_amd_simulation.context.setPositions(pdb.positions)
for i in range(10):
    lang_amd_simulation.step(10)
    n_steps = str(lang_amd_simulation.context.getStepCount())
    deltaV_val = str(lang_amd_integrator.getGlobalVariable(2))
    v0 = str(lang_amd_integrator.getGlobalVariable(3))
    d1, d2 = dist_measurer.getCollectiveVariableValues(lang_amd_simulation.context)
    print("%s %s %s %s %s" % (n_steps, v0, deltaV_val, str(d1), str(d2)))

10 -105376.95061972504 31921.666081284708 0.4592479467391968 0.42698609828948975
20 -95968.91936972504 23155.1888257363 0.4693585932254791 0.4319298267364502
30 -88821.95061972504 16705.799805241553 0.46689820289611816 0.43246862292289734
40 -84715.82561972504 13132.032521123332 0.4591708481311798 0.4342583417892456
50 -81850.95061972504 10721.01868945394 0.4615037143230438 0.43215373158454895
60 -79956.57561972504 9175.14567464759 0.4560432732105255 0.4356312155723572
70 -78998.51311972504 8410.938899910136 0.462199866771698 0.444046288728714
80 -78531.38811972504 8043.127945221054 0.4730907678604126 0.4438447058200836
90 -77720.45061972504 7412.678721729875 0.47484129667282104 0.4424886703491211
100 -78050.76311972504 7668.1946981427345 0.4741080105304718 0.4484359920024872
